### Code to Preprocess UKB Raw Dataset

In [ ]:
# import library
import pyspark
from pyspark.sql import SparkSession

# define data path
path_in = "../../../ukb51139_v2.csv"
path_out = "../../../ukb_subset_v2.csv"

# set up PySpark session
spark = SparkSession.builder.appName("UKB").getOrCreate()

### Load and Save Dataset

In [ ]:
# Load ukb as spark data frame
df = spark.read.csv(path_in, header=True)

# Filter missing blood pressure rows
df = df.filter("(`X4080.2.0` is NOT NULL OR `X93.2.0` is NOT NULL) AND \
                (`X4079.2.0` is NOT NULL OR `X94.2.0` is NOT NULL)")

# Show blood pressure columns
df.select(df["`X4080.2.0`"], df["`X4079.2.0`"], 
          df["`X93.2.0`"], df["`X94.2.0`"]).limit(10).show()

# Show number of rows
#print("Number of rows after initial filtering is " + str(df.count()))

# Save data to file
#df.write.option(header = "True", delimiter = ',').csv(path_out)

### Use SQL Queries

In [ ]:
# register data frame as a sql table
df.registerTempTable('ukb')

# run SQL queries as normal
spark.sql('SELECT AVG(`X4080.2.0`) FROM ukb WHERE `X31.0.0` = 1').show()